<a href="https://colab.research.google.com/github/puneet170191/Roadmap-To-Learn-Generative-AI-In-2024/blob/main/V1_DeepSPEC_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
pip install PyWavelets

Multiscale CNN wtih PHysics informed features no transformer

In [18]:
# DeepSPEC: Hybrid Speckle Classification Model with Per-Sample Physics Guidance
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from einops import rearrange
from glob import glob
from PIL import Image
import os
import numpy as np
import random

# Reproducibility
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
np.random.seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Root directory and preview
root_dir = "/content/drive/My Drive/00_DeepSPEC/DataSet_Learning/"
print("Classes:", os.listdir(root_dir))
for class_id in range(7):
    class_dir = os.path.join(root_dir, str(class_id))
    sample_imgs = glob(f"{class_dir}/**/*.tiff", recursive=True)
    print(f"Class {class_id} has {len(sample_imgs)} images")

# Utility: Compute Speckle Features (mocked for now)
def compute_speckle_features(img_tensor):
    return torch.rand(16)  # Placeholder for real speckle feature extraction

# 1. Multi-Scale CNN with 4 filters
class MultiScaleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_gaussian = nn.Conv2d(1, 8, kernel_size=3, padding=1)
        self.conv_laplacian = nn.Conv2d(1, 8, kernel_size=3, padding=1)
        self.conv_wavelet = nn.Conv2d(1, 8, kernel_size=3, padding=1)
        self.conv_atrous = nn.Conv2d(1, 8, kernel_size=3, padding=2, dilation=2)
        self.norm = nn.BatchNorm2d(32)
        self.relu = nn.ReLU()

    def forward(self, x):
        g = self.conv_gaussian(x)
        l = self.conv_laplacian(x)
        w = self.conv_wavelet(x)
        a = self.conv_atrous(x)
        out = torch.cat([g, l, w, a], dim=1)
        out = self.relu(self.norm(out))
        return out

# 2. Patch Embedding Layer
class PatchEmbed(nn.Module):
    def __init__(self, in_channels=32, patch_size=50, emb_dim=128):
        super().__init__()
        self.proj = nn.Conv2d(in_channels, emb_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        x = self.proj(x)
        x = x.flatten(2).transpose(1, 2)
        return x

# 3. Physics-Informed Transformer Block
class PhysicsTransformerBlock(nn.Module):
    def __init__(self, dim, nhead=4):
        super().__init__()
        self.attn = nn.MultiheadAttention(embed_dim=dim, num_heads=nhead, batch_first=True)
        self.norm1 = nn.LayerNorm(dim)
        self.norm2 = nn.LayerNorm(dim)
        self.ff = nn.Sequential(nn.Linear(dim, dim * 4), nn.ReLU(), nn.Linear(dim * 4, dim))

    def forward(self, x, physics_embedding):
        physics_embedding = physics_embedding.unsqueeze(1).expand(-1, x.size(1), -1)
        x = x + physics_embedding
        attn_out, _ = self.attn(x, x, x)
        x = self.norm1(x + attn_out)
        x = self.norm2(x + self.ff(x))
        return x

# 4. DeepSPEC Model
class DeepSPEC(nn.Module):
    def __init__(self, num_classes=7):
        super().__init__()
        self.phys_projector = nn.Linear(16, 128)
        self.cnn = MultiScaleCNN()
        self.patch_embed = PatchEmbed()
        self.transformer = PhysicsTransformerBlock(128)
        self.classifier = nn.Sequential(nn.LayerNorm(128), nn.Linear(128, num_classes))

    def forward(self, x, physics_feat):
        cnn_feat = self.cnn(x)
        patches = self.patch_embed(cnn_feat)
        physics_emb = self.phys_projector(physics_feat)
        transformer_out = self.transformer(patches, physics_emb)
        cls_token = transformer_out.mean(dim=1)
        out = self.classifier(cls_token)
        return out, physics_feat

# 5. Dataset with per-sample feature extraction
class DeepSpecDataset(Dataset):
    def __init__(self, root_dir, img_size=350, samples_per_class=None):
        self.root_dir = root_dir
        self.img_size = img_size
        self.images_path = []
        self.labels = []
        class_folders = sorted(glob(f"{root_dir}/*"))
        for class_path in class_folders:
            class_idx = int(os.path.basename(class_path))
            sub_folders = glob(f"{class_path}/*")
            class_images = []
            for sub_folder in sub_folders:
                class_images.extend(glob(f"{sub_folder}/*.tiff"))
            if samples_per_class:
                class_images = random.sample(class_images, min(samples_per_class, len(class_images)))
            self.images_path.extend(class_images)
            self.labels.extend([class_idx] * len(class_images))

        self.transform = transforms.Compose([
            transforms.Grayscale(),
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor()
        ])

    def __len__(self):
        return len(self.images_path)

    def __getitem__(self, idx):
        img = Image.open(self.images_path[idx])
        img_tensor = self.transform(img)
        label = self.labels[idx]
        physics_feat = compute_speckle_features(img_tensor)
        return img_tensor, label, physics_feat

# 6. Loss Function

def hybrid_loss(outputs, targets, physics_feats, lambda_reg=0.01):
    ce_loss = F.cross_entropy(outputs, targets)
    phys_reg = torch.mean(torch.abs(physics_feats))
    return ce_loss + lambda_reg * phys_reg

# 7. Training Loop

def train_model(model, train_loader, val_loader, epochs=10, lr=1e-3, device='cuda'):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.to(device)

    for ep in range(epochs):
        model.train()
        total, correct, total_loss = 0, 0, 0.0
        for imgs, labels, physics_feats in train_loader:
            imgs, labels, physics_feats = imgs.to(device), labels.to(device), physics_feats.to(device)
            optimizer.zero_grad()
            outputs, phys = model(imgs, physics_feats)
            loss = hybrid_loss(outputs, labels, phys)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_acc = 100. * correct / total
        print(f"Epoch {ep+1}/{epochs}: Train Loss={total_loss/len(train_loader):.4f}, Train Acc={train_acc:.2f}%")

        model.eval()
        val_correct, val_total, val_loss = 0, 0, 0.0
        all_preds, all_labels = [], []
        with torch.no_grad():
            for imgs, labels, physics_feats in val_loader:
                imgs, labels, physics_feats = imgs.to(device), labels.to(device), physics_feats.to(device)
                outputs, phys = model(imgs, physics_feats)
                loss = hybrid_loss(outputs, labels, phys)
                val_loss += loss.item()
                preds = outputs.argmax(dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        val_acc = 100. * val_correct / val_total
        print(f"Epoch {ep+1}/{epochs}: Val Loss={val_loss/len(val_loader):.4f}, Val Acc={val_acc:.2f}%")

    print("\nFinal Evaluation Metrics:")
    print(confusion_matrix(all_labels, all_preds))
    print(classification_report(all_labels, all_preds, digits=4))

# 8. Main Execution Script
if __name__ == '__main__':
    print("[INFO] Preparing dataset with per-sample physics features...")
    full_dataset = DeepSpecDataset(root_dir, samples_per_class=10)
    indices = list(range(len(full_dataset)))
    train_idx, val_idx = train_test_split(indices, test_size=0.2, random_state=SEED, stratify=[full_dataset.labels[i] for i in indices])

    print(f"[INFO] Training samples: {len(train_idx)}, Validation samples: {len(val_idx)}")
    train_dataset = Subset(full_dataset, train_idx)
    val_dataset = Subset(full_dataset, val_idx)

    print("[INFO] Creating data loaders...")
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

    print("[INFO] Initializing model and starting training...")
    model = DeepSPEC(num_classes=7)
    train_model(model, train_loader, val_loader, epochs=10, lr=1e-3, device='cuda' if torch.cuda.is_available() else 'cpu')

    print("[INFO] Training complete.")

Classes: ['0', '1', '2', '3', '4', '5', '6']
Class 0 has 23602 images
Class 1 has 22009 images
Class 2 has 21505 images
Class 3 has 7125 images
Class 4 has 7125 images
Class 5 has 5281 images
Class 6 has 5291 images
[INFO] Preparing dataset with per-sample physics features...
[INFO] Training samples: 56, Validation samples: 14
[INFO] Creating data loaders...
[INFO] Initializing model and starting training...
Epoch 1/10: Train Loss=2.9922, Train Acc=14.29%
Epoch 1/10: Val Loss=2.8395, Val Acc=14.29%
Epoch 2/10: Train Loss=2.3317, Train Acc=16.07%
Epoch 2/10: Val Loss=2.0734, Val Acc=21.43%
Epoch 3/10: Train Loss=1.7836, Train Acc=32.14%
Epoch 3/10: Val Loss=1.8934, Val Acc=14.29%
Epoch 4/10: Train Loss=1.8120, Train Acc=28.57%
Epoch 4/10: Val Loss=1.9004, Val Acc=14.29%
Epoch 5/10: Train Loss=1.8528, Train Acc=28.57%
Epoch 5/10: Val Loss=1.8772, Val Acc=28.57%
Epoch 6/10: Train Loss=1.7723, Train Acc=35.71%
Epoch 6/10: Val Loss=1.7936, Val Acc=35.71%
Epoch 7/10: Train Loss=1.7161, Train

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
